# Seyed Mohammad Amin Atyabi - 830402014 - HW 2

In [ ]:
from copy import deepcopy

EMPTY = 0


def next_state(current):
    new_states = []
    for i in range(len(current)):
        for j in range(len(current[i])):
            if j + 1 < len(current[i]) and current[i][j + 1] == EMPTY:
                new_state = deepcopy(current)
                new_state[i][j] = EMPTY
                new_state[i][j + 1] = current[i][j]
                new_states.append(new_state)
            elif j - 1 > 0 and current[i][j - 1] == EMPTY:
                new_state = deepcopy(current)
                new_state[i][j] = EMPTY
                new_state[i][j - 1] = current[i][j]
                new_states.append(new_state)
            elif i + 1 < len(current[j]) and current[i + 1][j] == EMPTY:
                new_state = deepcopy(current)
                new_state[i][j] = EMPTY
                new_state[i + 1][j] = current[i][j]
                new_states.append(new_state)
            elif i - 1 > 0 and current[i - 1][j] == EMPTY:
                new_state = deepcopy(current)
                new_state[i][j] = EMPTY
                new_state[i - 1][j] = current[i][j]
                new_states.append(new_state)
    return new_states


def print_state(state):
    for i in state:
        print(str(i).replace('[', '').replace(']', '').replace(',', ' |').replace('EMPTY', ' '))


def find_index(state, number):
    for i in range(len(state)):
        for j in range(len(state[i])):
            if state[i][j] == number:
                return i, j
    return -1, -1


def manhattan_distance(state_1, state_2):
    distance = 0
    for i in range(len(state_1)):
        for j in range(len(state_1[i])):
            if state_1[i][j] != EMPTY:
                dis_i, dis_j = find_index(state_2, state_1[i][j])
                distance += (abs(i - dis_i) + abs(j - dis_j))
    return distance

state_1 = [
    [1, 2, 3],
    [4, 5, 6],
    [7, 8, 0]
]

print_state(state_1)
next = next_state(state_1)
print()
print()
for n in next:
    print_state(n)
    print()
    print()

state_2 = [
    [1, 2, 3],
    [4, 0, 5],
    [6, 7, 8]
]

print_state(state_2)
next = next_state(state_2)
print()
print()
for n in next:
    print_state(n)
    print()
    print()

print()
print()
print(manhattan_distance(state_1, state_2))